In [2]:
import pandas as pd
import sparklines as sl

In [5]:
sl.sparklines(range(10),)

['▁▂▃▃▄▅▆▆▇█']

In [6]:
url = 'https://raw.githubusercontent.com/mattharrison/datasets/master/data/dirtydevil.txt'
df = pd.read_csv(url,skiprows=lambda x: x<34 or x ==35 ,sep ='\t')
df.head()

/var/folders/pg/zck9hy351gbfrqfqp_z4gb1m0000gn/T/ipykernel_1912/633917389.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url,skiprows=lambda x: x<34 or x ==35 ,sep ='\t')


,agency_cd,site_no,datetime,tz_cd,144166_00060,144166_00060_cd,144167_00065,144167_00065_cd
0,USGS,9333500,2001-05-07 01:00,MDT,71.0,A:[91],NaN,NaN
1,USGS,9333500,2001-05-07 01:15,MDT,71.0,A:[91],NaN,NaN
2,USGS,9333500,2001-05-07 01:30,MDT,71.0,A:[91],NaN,NaN
3,USGS,9333500,2001-05-07 01:45,MDT,70.0,A:[91],NaN,NaN
4,USGS,9333500,2001-05-07 02:00,MDT,70.0,A:[91],NaN,NaN


In [9]:

def tweak_river (df_ ):
    return (df_
. assign ( datetime = pd.to_datetime(df_.datetime) )
. rename ( columns ={ '144166_00060': 'cfs','144167_00065': 'gage_height'})
.set_index('datetime')
)

In [11]:
dd = tweak_river(df)
dd

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
datetime,,,,,,,
2001-05-07 01:00:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:15:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:30:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:45:00,USGS,9333500,MDT,70.00,A:[91],NaN,NaN
2001-05-07 02:00:00,USGS,9333500,MDT,70.00,A:[91],NaN,NaN
...,...,...,...,...,...,...,...
2020-09-28 08:30:00,USGS,9333500,MDT,9.53,P,6.16,P
2020-09-28 08:45:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 09:00:00,USGS,9333500,MDT,9.20,P,6.15,P


In [44]:
agg_flow = (dd
.groupby(pd.Grouper(freq='M'))
.agg(cfs = ('cfs','median'),
    total_flow = ('cfs',lambda ser :( ser *15*60).sum()),
    gage_height = ('gage_height','median'),
 flow_trend =('cfs', lambda ser: sl.sparklines(
 ser
 .fillna(0)
 .resample('2D')
 .median()
 .fillna(0))
 [0]
 ))
 .assign(quarterly_flow = lambda df_: df_.total_flow
    .resample('Q')
    .transform('sum')
    ,percecnt_quarerly_flow = lambda df2_: (df2_.total_flow / df2_.quarterly_flow)
    ,off_goal = lambda df3_: df3_.percecnt_quarerly_flow - 0.33
    ,cost = lambda df4_: df4_.total_flow * 0.0002
 ))
agg_flow.head()

,cfs,total_flow,gage_height,flow_trend,quarterly_flow,percecnt_quarerly_flow,off_goal,cost
datetime,,,,,,,,
2001-05-31,47.0,105383700.0,NaN,█▅▅▃▅▄▄▃▂▃▂▁▁,123227100.0,0.855199,0.525199,21076.740
2001-06-30,23.0,17843400.0,NaN,█▇▆▄▁▁▁▁▁▁▁▁▁▁▆,123227100.0,0.144801,-0.185199,3568.680
2001-07-31,17.0,7781400.0,NaN,▁█▄,243448920.0,0.031963,-0.298037,1556.280
2001-08-31,52.5,192848220.0,NaN,█▂▁▁▁,243448920.0,0.792151,0.462151,38569.644
2001-09-30,26.0,42819300.0,NaN,▆▆▁▁▅▆▁██▆▆▆▆,243448920.0,0.175886,-0.154114,8563.860


In [50]:
(agg_flow
.reset_index()
.style
.format({'total_flow': '{:,.0f}'
,'cost': '${:,.2f}' ,'percecnt_quarerly_flow': '{:,.0%}'
}
,na_rep='Missing')
.bar(subset=['cfs'],color='#d62728',vmax=agg_flow.cfs.quantile(.95))
)


,datetime,cfs,total_flow,gage_height,flow_trend,quarterly_flow,percecnt_quarerly_flow,off_goal,cost
0,2001-05-31 00:00:00,47.000000,"105,383,700",Missing,█▅▅▃▅▄▄▃▂▃▂▁▁,123227100.000000,86%,0.525199,"$21,076.74"
1,2001-06-30 00:00:00,23.000000,"17,843,400",Missing,█▇▆▄▁▁▁▁▁▁▁▁▁▁▆,123227100.000000,14%,-0.185199,"$3,568.68"
2,2001-07-31 00:00:00,17.000000,"7,781,400",Missing,▁█▄,243448920.000000,3%,-0.298037,"$1,556.28"
3,2001-08-31 00:00:00,52.500000,"192,848,220",Missing,█▂▁▁▁,243448920.000000,79%,0.462151,"$38,569.64"
4,2001-09-30 00:00:00,26.000000,"42,819,300",Missing,▆▆▁▁▅▆▁██▆▆▆▆,243448920.000000,18%,-0.154114,"$8,563.86"
5,2001-10-31 00:00:00,54.000000,"134,975,700",Missing,▁▁▂▂▃▄▄▅▅▅▆▇▇▇█,480483900.000000,28%,-0.049084,"$26,995.14"
6,2001-11-30 00:00:00,104.000000,"203,110,200",Missing,▆▆▆▆▆▇▇▇██▁▁▁▇▇,480483900.000000,42%,0.092720,"$40,622.04"
7,2001-12-31 00:00:00,115.000000,"142,398,000",Missing,▆▁▁▁▇▇▁▁▁▆▆▆▁▆▇█,480483900.000000,30%,-0.033636,"$28,479.60"
8,2002-01-31 00:00:00,136.000000,"197,745,300",Missing,██▇███▇▇▁▁▁▁▁▁▇,638525700.000000,31%,-0.020310,"$39,549.06"
9,2002-02-28 00:00:00,131.000000,"157,920,300",Missing,▄▄█▆▆█▁,638525700.000000,25%,-0.082680,"$31,584.06"
